<a href="https://colab.research.google.com/github/FranciscoReveriano/Facial-Expression-Recognition/blob/Version-1-(50-Epochs)/Copy_of_train_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Original Model

##Train The Dataset

### Import Necessary Libaries

In [1]:
import tensorflow as tf
import numpy as np
import keras
import time
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


###Check for GPU

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

config = tf.ConfigProto()
sess = tf.Session(config=config)
keras.backend.set_session(sess)

Found GPU at: /device:GPU:0


### Select Settings

In [0]:
num_classes = 7
batch_size = 256
epochs = 50

### Process Images

In [4]:
start = time.time() 
with tf.device('/GPU:0'):
  with open("fer2013.csv") as f:
      content = f.readlines()
  lines = np.array(content)
  num_of_instances = lines.size
end = time.time()
print("Number of Images: ", num_of_instances)
print("Pixel number of each image: ", len(lines[1].split(",")[1].split(" ")))
print("Time to prepare images = ", end-start)

Number of Images:  35888
Pixel number of each image:  2304
Time to prepare images =  2.2839956283569336


### Train the Model

In [5]:
start = time.time()
x_train, y_train = [], []
with tf.device('/GPU:0'):
  for i in range(1, num_of_instances):
      emotion, img, usage = lines[i].split(",")
      val = img.split(" ")
      pixels = np.array(val, 'float32')
      emotion = keras.utils.to_categorical(emotion, num_classes)
      if 'Training' in usage:
          y_train.append(emotion)
          x_train.append(pixels)

  x_train = np.array(x_train, 'float32')
  y_train = np.array(y_train, 'float32')
  x_train /= 255
  x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
  x_train = x_train.astype('float32')
  print(x_train.shape[0], 'train samples')

  model = Sequential()

  model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48, 48, 1)))
  model.add(MaxPooling2D(pool_size=(5, 5), strides=(2, 2)))

  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))

  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))

  model.add(Flatten())

  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.2))

  model.add(Dense(num_classes, activation='softmax'))

  gen = ImageDataGenerator()
  train_generator = gen.flow(x_train, y_train, batch_size=batch_size)
  model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
  model.fit_generator(train_generator, steps_per_epoch=batch_size, epochs=epochs)

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
end = time.time()
print("Saved model to disk")
print("Total Runtime To Train Model = ", end-start)

28709 train samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
256/256 [==============================] - 11s 44ms/step - loss: 1.7950 - acc: 0.2561
Epoch 2/50
256/256 [==============================] - 7s 27ms/step - loss: 1.5608 - acc: 0.3813
Epoch 3/50
256/256 [==============================] - 7s 27ms/step - loss: 1.3964 - acc: 0.4626
Epoch 4/50
256/256 [==============================] - 7s 27ms/step - loss: 1.2960 - acc: 0.5037
Epoch 5/50
256/256 [==============================] - 7s 27ms/step - loss: 1.1998 - acc: 0.5438
Epoch 6/50
256/256 [==============================] - 7s 28ms/step - loss: 1.1331 - acc: 0.5680
Epoch 7/50
256/256 [==============================] - 7s 28ms/step - loss: 1.0604 - acc: 0.5980
Epoch 8/50
256/256 [==============================] - 7s 28ms/step - los

##Test the Dataset

###Import Necessary Libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import time
from keras.models import model_from_json

In [7]:
start = time.time()
num_classes = 7
with tf.device('/GPU:0'):
  with open("fer2013.csv") as f:
      content = f.readlines()
  lines = np.array(content)
  num_of_instances = lines.size

  x_test, y_test = [], []

  for i in range(1, num_of_instances):
      emotion, img, usage = lines[i].split(",")
      val = img.split(" ")
      pixels = np.array(val, 'float32')
      emotion = keras.utils.to_categorical(emotion, num_classes)
      if 'PublicTest' in usage:
          y_test.append(emotion)
          x_test.append(pixels)

  x_test = np.array(x_test, 'float32')
  y_test = np.array(y_test, 'float32')
  x_test /= 255
  x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
  x_test = x_test.astype('float32')
  print(x_test.shape[0], 'test samples')

  json_file = open('model.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)
  loaded_model.load_weights("model.h5")
  print("Loaded model from disk")
  loaded_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])


def emotion_analysis(emotions):
    objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, emotions, align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    plt.show()

evaluation = loaded_model.evaluate(x_test, y_test, batch_size=256, verbose=1)
end = time.time()
print('Summary: Loss over the test data set: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))
print("Total Time in Testing Model: ", end-start)

3589 test samples
Loaded model from disk
3589/3589 [==============================] - 0s 92us/step
Summary: Loss over the test data set: 3.73, Accuracy: 0.57
Total Time in Testing Model:  15.731481313705444
